In [42]:
library(sp)
library(rgdal)

library(ggplot2)
library(ggmap)
library(Rcpp)

ERROR: Error in library(rgdal): there is no package called ‘rgdal’


In [45]:
# install.packages("raster")
# install.packages("sp", dependencies = F)
install.packages("/Users/sam/Downloads/rgdal_1.2-7.tar",repos = NULL, type = .Platform$pkgType)

Warning message in install.packages("/Users/sam/Downloads/rgdal_1.2-7.tar", repos = NULL, :
“installation of package ‘/Users/sam/Downloads/rgdal_1.2-7.tar’ had non-zero exit status”

In [39]:
getClass('Spatial')

Class "Spatial" [package "sp"]

Slots:
                              
Name:         bbox proj4string
Class:      matrix         CRS

Known Subclasses: 
Class "SpatialPoints", directly
Class "SpatialMultiPoints", directly
Class "SpatialGrid", directly
Class "SpatialLines", directly
Class "SpatialPolygons", directly
Class "SpatialPointsDataFrame", by class "SpatialPoints", distance 2
Class "SpatialPixels", by class "SpatialPoints", distance 2
Class "SpatialMultiPointsDataFrame", by class "SpatialMultiPoints", distance 2
Class "SpatialGridDataFrame", by class "SpatialGrid", distance 2
Class "SpatialLinesDataFrame", by class "SpatialLines", distance 2
Class "SpatialPixelsDataFrame", by class "SpatialPoints", distance 3
Class "SpatialPolygonsDataFrame", by class "SpatialPolygons", distance 2

In [6]:
getClass('CRS')

Class "CRS" [package "sp"]

Slots:
                
Name:   projargs
Class: character

In [13]:
m <- matrix(c(0, 0, 1, 1), # Create a list with 4 elements
            ncol = 2,      # Make two column matrix
            dimnames = list(NULL, c("min", "max"))) # Provide dimension name to the matrix
m
crs <- CRS(projargs = as.character(NA))
crs
S <- Spatial(bbox = m, proj4string = crs)
S

min,max
0,1
0,1


CRS arguments: NA 

An object of class "Spatial"
Slot "bbox":
     min max
[1,]   0   1
[2,]   0   1

Slot "proj4string":
CRS arguments: NA 


In [12]:
list(NULL, c("min", "max"))

[[1]]
NULL

[[2]]
[1] "min" "max"

In [15]:
install.packages('Rcpp')

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


# PART 2: CRAN Data 

In [24]:
library(sp)
library(ggplot2)
library(ggmap)
library(Rcpp)

library(maps)
library(maptools)  # For map2SpatialLines function

In [14]:
data(crime)
crimeDF <- crime
dim(crimeDF)
head(crimeDF)
str(crimeDF)
summary(crimeDF)

[1] 86314    17

,time,date,hour,premise,offense,beat,block,street,type,suffix,number,month,day,location,address,lon,lat
82729,2010-01-01,1/1/2010,0,18A,murder,15E30,9600-9699,marlive,ln,-,1,january,friday,apartment parking lot,9650 marlive ln,-95.43739,29.67790
82730,2010-01-01,1/1/2010,0,13R,robbery,13D10,4700-4799,telephone,rd,-,1,january,friday,road / street / sidewalk,4750 telephone rd,-95.29888,29.69171
82731,2010-01-01,1/1/2010,0,20R,aggravated assault,16E20,5000-5099,wickview,ln,-,1,january,friday,residence / house,5050 wickview ln,-95.45586,29.59922
82732,2010-01-01,1/1/2010,0,20R,aggravated assault,2A30,1000-1099,ashland,st,-,1,january,friday,residence / house,1050 ashland st,-95.40334,29.79024
82733,2010-01-01,1/1/2010,0,20A,aggravated assault,14D20,8300-8399,canyon,,-,1,january,friday,apartment,8350 canyon,-95.37791,29.67063
82734,2010-01-01,1/1/2010,0,20R,burglary,18F60,9300-9399,rowan,ln,-,1,january,friday,residence / house,9350 rowan ln,-95.54830,29.70223


'data.frame':	86314 obs. of  17 variables:
 $ time    : POSIXt, format: "2010-01-01 00:00:00" "2010-01-01 00:00:00" ...
 $ date    : chr  "1/1/2010" "1/1/2010" "1/1/2010" "1/1/2010" ...
 $ hour    : int  0 0 0 0 0 0 0 0 0 0 ...
 $ premise : chr  "18A" "13R" "20R" "20R" ...
 $ offense : Factor w/ 7 levels "aggravated assault",..: 4 6 1 1 1 3 3 3 3 3 ...
 $ beat    : chr  "15E30" "13D10" "16E20" "2A30" ...
 $ block   : chr  "9600-9699" "4700-4799" "5000-5099" "1000-1099" ...
 $ street  : chr  "marlive" "telephone" "wickview" "ashland" ...
 $ type    : chr  "ln" "rd" "ln" "st" ...
 $ suffix  : chr  "-" "-" "-" "-" ...
 $ number  : int  1 1 1 1 1 1 1 1 1 1 ...
 $ month   : Ord.factor w/ 8 levels "january"<"february"<..: 1 1 1 1 1 1 1 1 1 1 ...
 $ day     : Ord.factor w/ 7 levels "monday"<"tuesday"<..: 5 5 5 5 5 5 5 5 5 5 ...
 $ location: chr  "apartment parking lot" "road / street / sidewalk" "residence / house" "residence / house" ...
 $ address : chr  "9650 marlive ln" "4750 telephone rd

      time                         date                hour      
 Min.   :2010-01-01 00:00:00   Length:86314       Min.   : 0.00  
 1st Qu.:2010-03-07 19:00:00   Class :character   1st Qu.: 8.00  
 Median :2010-05-06 07:00:00   Mode  :character   Median :14.00  
 Mean   :2010-05-04 13:47:44                      Mean   :13.28  
 3rd Qu.:2010-07-02 15:00:00                      3rd Qu.:19.00  
 Max.   :2010-08-31 23:00:00                      Max.   :23.00  
                                                                 
   premise                        offense          beat          
 Length:86314       aggravated assault: 7177   Length:86314      
 Class :character   auto theft        : 7946   Class :character  
 Mode  :character   burglary          :17802   Mode  :character  
                    murder            :  157                     
                    rape              :  378                     
                    robbery           : 6298                     
          

#### ** A Spatial DataFrame has five components or slots: **
    
   * data: This is the data contained in the data Frame.
   * coords.nrs: This is the column position for the coordinate data. For our case we use we use the last two columns as coordinate system and so coorsa.nrs would have column placement for those two column position.
   * coords: This are the coordinates (Lat and Long)
   * bbox: The bounding box of the coordinate
   * proj4String: Gives us the coordinate reference system CRS()
   
   
#### ** Coordinate Reference System (CRS): **
   * A coordinate reference system is of two types {Geographic Coordinates and Projected coordinates}
       * Geographic Coordiante reference system (LAT and LONG). This represents the earth as a sphere, For example : WGS84 is represented as (33.33, 44.4)
       * Cartesian/Projected coordinate system (x,y), This coordinate sytems represents earth as a flat surface with distances measures in meters or Kilometers. So we can say that the earth's geographical coordinates (of elipsoid earth) are projected into a planer system (plane surface). Under the Planner system we have several types of *Coordinate Reference Systems* Two common once are given
           1. State Planer (NADS83) (Mostely used in United States), Ex, Nevada State Plane NADS83
           2. Universal Transverse Merctor (Across the universe), Ex, Nevada uses (UTM Zone 11N)
           
#### ** Spatial Polygon DataFrame: **
   * We can get shape file polygons from different sources
       * Data Sources 1: US census website
       * Free Spatial data by country from DIVA-GIS website

In [35]:
dataDir <- '/Users/sam/All-Program/App-DataSet/Study/GeoSpatial-Analysis/CRAN051001.txt'
# Load a geospatial dataset
cranDF <- read.table(dataDir, header = TRUE)
summary(cranDF)

            place        north          east             loc    
 Bern          : 2   40d26'N: 2   13d22'E : 2   Brazil     : 5  
 Pittsburgh, PA: 2   46d57'N: 2   7d26'E  : 2   Germany    : 5  
 Aalborg       : 1   20d45'S: 1   80d0'W  : 2   Italy      : 4  
 Aizu          : 1   22d43'S: 1   0d10'W  : 1   France     : 3  
 Ames, IA      : 1   22d54'S: 1   118d15'W: 1   Switzerland: 3  
 Arezzo        : 1   23d32'S: 1   118d47'E: 1   Australia  : 2  
 (Other)       :46   (Other):46   (Other) :45   (Other)    :32  
      long                lat        
 Min.   :-122.9500   Min.   :-37.82  
 1st Qu.: -47.3833   1st Qu.: 34.52  
 Median :   7.8500   Median : 42.73  
 Mean   :  -0.6617   Mean   : 31.73  
 3rd Qu.:  16.8333   3rd Qu.: 47.58  
 Max.   : 153.0333   Max.   : 57.05  
                                     

In [16]:
# We should specify to R that we have a latitude and logitude coordinate system
llCRS <- CRS('+proj-longlat +ellps=WG584')

# cranDF is a local R dataframe, Most of the spatial packages
# do not inderstand the local R data frame, so here we convert 
# them into spatial object
coords <- SpatialPoints(cranDF[, c('long', 'lat')], proj4string = llCRS)  
cranSpatial_DF <- SpatialPointsDataFrame(coords, cranDF) # Creating a spatial data frame, we explicitely specify that coords
summary(cranSpatial_DF)

# Here in the above code, we tell "sp" that the two columns 'lon' and 'lat' are the actual coordinates. 
# And then while creating the dataframe we explicitely add the "coords" with all other column. 
# When we specify "coords" "sp" understands that it has to create a column "coordinates" with two fields
# Lat and Lon, so that the coordinates can be consumed by other methods easily

# The argument in the summary that states "Is projected=False" states that the 
# coordinate points are not planner (which means they are not projected) on a plane

Object of class SpatialPointsDataFrame
Coordinates:
            min      max
long -122.95000 153.0333
lat   -37.81667  57.0500
Is projected: FALSE 
proj4string : [+proj-longlat +ellps=WG584]
Number of points: 54
Data attributes:
            place        north          east             loc    
 Bern          : 2   40d26'N: 2   13d22'E : 2   Brazil     : 5  
 Pittsburgh, PA: 2   46d57'N: 2   7d26'E  : 2   Germany    : 5  
 Aalborg       : 1   20d45'S: 1   80d0'W  : 2   Italy      : 4  
 Aizu          : 1   22d43'S: 1   0d10'W  : 1   France     : 3  
 Ames, IA      : 1   22d54'S: 1   118d15'W: 1   Switzerland: 3  
 Arezzo        : 1   23d32'S: 1   118d47'E: 1   Australia  : 2  
 (Other)       :46   (Other):46   (Other) :45   (Other)    :32  
      long                lat        
 Min.   :-122.9500   Min.   :-37.82  
 1st Qu.: -47.3833   1st Qu.: 34.52  
 Median :   7.8500   Median : 42.73  
 Mean   :  -0.6617   Mean   : 31.73  
 3rd Qu.:  16.8333   3rd Qu.: 47.58  
 Max.   : 153.0333   Ma

In [17]:
# The bbox method provides a bounding box of the object.
# Basically it provides the minimum and maximum boundaries.
bbox(cranSpatial_DF)

# While creating the data frame we initialize te column coordinate.
# Let us now get the coordinates where murder was the crime motive using the 
# coordinate column
brazil <- which(cranSpatial_DF$loc == 'Brazil')
brazilCoordinates <- coordinates(cranSpatial_DF)[brazil, ]
head(brazilCoordinates)

# We can also do the above by simply using spatial Dataframe
# head(crimeSpatial_DF[murder, ])

,min,max
long,-122.95000,153.0333
lat,-37.81667,57.0500


long,lat
-49.26667,-25.41667
-42.86667,-20.75000
-43.20000,-22.90000
-47.63333,-22.71667
-46.63333,-23.53333


## Understanding Spatial Lines:
------

  * Lines are simple sequence of points. 
  * How a line is represented in the Map package:
      * Given the coordinates we start from one point and continue the sequence until a NA is reached. For Example, The x coordinate (longitute) of a particular country might have a sequence of points such as 123.8887, 123.8256, 123.7498, 123.6807, 123.6798, 123.7523, 123.7537, 123.7715, 123.9349, 123.9281, 123.8887, **NA**, 124.2932, 124.2343, 124.1856, 124.1357, 124.0848, 124.1204, 124.1702, 124.2105, 124.3020, 124.3240, 124.2932, **NA**, as so on.....
          
          In such a case we draw a line from a point untill a **NA** is reached. And for the next line we start from the immediate non **NA** points and continue the trend.
          
### Spatial Line Object ():

  * A SpatialLine object is a matrix of 2D coordinates, without NA values
  * Example : the coordinate points for lines from **Map** package may look like 
  
      x (Long) : 123.8887, 123.8256, 123.7498, 123.6807, 123.6798, 123.7523, 123.7537, 123.7715, 123.9349, 123.9281, 123.8887, **NA**, 124.2932, 124.2343, 124.1856, 124.1357, 124.0848, 124.1204, 124.1702, 124.2105, 124.3020, 124.3240, 124.2932, **NA**
  
      y (Lat)  : 24.28013, 24.26606, 24.28330, 24.28804, 24.31777, 24.34849, 24.39131, 24.41445, 24.36201, 24.32363, 24.28013, **NA**, 24.51592, 24.35806, 24.33506, 24.34761, 24.43584, 24.46963, 24.45185, 24.45864, 24.58711, 24.56636, 24.51592, **NA**
      
  * The Coordinate points for lines when converted to SpatialLines are packed into a dataframe and look like,  
      * Slot "ID": "1":
          1. x (Long) : 123.8887, 123.8256, 123.7498, 123.6807, 123.6798, 123.7523, 123.7537, 123.7715, 123.9349, 123.9281, 123.8887
          2. y (Lat)  : 24.28013, 24.26606, 24.28330, 24.28804, 24.31777, 24.34849, 24.39131, 24.41445, 24.36201, 24.32363, 24.28013
         
      * Slot "ID": "2":
          1. x (Long) : 124.2932, 124.2343, 124.1856, 124.1357, 124.0848, 124.1204, 124.1702, 124.2105, 124.3020, 124.3240, 124.2932
          2. y (Lat)  : 24.51592, 24.35806, 24.33506, 24.34761, 24.43584, 24.46963, 24.45185, 24.45864, 24.58711, 24.56636, 24.51592

In [47]:
# getClass("SpatialLines")

japan <- map("world", "japan", plot = FALSE)
# Japan here is a list of all the x(Longitude), y(Latitude) coordinate of the boundary of the country

# We initialize the projection to be Lat and Long or geospatial coordinate system
p4s <- CRS("+proj=longlat +ellps=WGS84")

# We retrieve the lines from the "map" package and convert them into a 
# spatialLine object using the map2SpatialLines of the "maptool" package
SLjapan <- map2SpatialLines(japan, proj4string = p4s)
# str(SLjapan, max.level = 2)

# Now that we have found out each slots of the lines, we wuld want to know the length of each slot 
# to have a high level understanding of the data
# slot(SLjapan, "lines")
Lines_len <- sapply(slot(SLjapan, "lines"),function(x) length(slot(x,"Lines")))
Lines_len
table(Lines_len)
    

[1] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

Lines_len
 1 
34 

In [55]:
nrow(volcano)
ncol(volcano)

[1] 87

[1] 61

In [50]:
volcano_sl <- ContourLines2SLDF(contourLines(volcano))
volcano_sl
# t(slot(volcano_sl, "data"))

                                        geometry level
C_1           MULTILINESTRING((1 0.5515289 ...))   100
C_2           MULTILINESTRING((0.1268403 0 ...))   110
C_3           MULTILINESTRING((0.1971154 0 ...))   120
C_4  MULTILINESTRING((0.04651163 0.5111111 ...))   130
C_5    MULTILINESTRING((0.06976744 0.49967 ...))   140
C_6  MULTILINESTRING((0.09302326 0.4944444 ...))   150
C_7       MULTILINESTRING((0.1162791 0.475 ...))   160
C_8   MULTILINESTRING((0.1395349 0.5151378 ...))   170
C_9   MULTILINESTRING((0.1627907 0.4984711 ...))   180
C_10  MULTILINESTRING((0.1976744 0.5318044 ...))   190

In [8]:
txsShape_datafile <- '/Users/sam/All-Program/App-DataSet/Study/GeoSpatial-Analysis/tl_2015_48_tract'
str(crimeSpatial_DF)
txsShape <- readOGR(dsn = "texasShape_datafile", layer = "tl_2015_48_tract")
class(txsShape)

Formal class 'SpatialPointsDataFrame' [package "sp"] with 5 slots
  ..@ data       :'data.frame':	33361 obs. of  17 variables:
  .. ..$ time    : POSIXt[1:33361], format: "2010-01-01 00:00:00" "2010-01-01 00:00:00" ...
  .. ..$ date    : Date[1:33361], format: "0001-01-20" "0001-01-20" ...
  .. ..$ hour    : int [1:33361] 0 0 0 0 0 0 0 0 0 0 ...
  .. ..$ premise : chr [1:33361] "18A" "13R" "20R" "20R" ...
  .. ..$ offense : Factor w/ 7 levels "aggravated assault",..: 4 6 1 1 1 3 3 3 3 3 ...
  .. ..$ beat    : chr [1:33361] "15E30" "13D10" "16E20" "2A30" ...
  .. ..$ block   : chr [1:33361] "9600-9699" "4700-4799" "5000-5099" "1000-1099" ...
  .. ..$ street  : chr [1:33361] "marlive" "telephone" "wickview" "ashland" ...
  .. ..$ type    : chr [1:33361] "ln" "rd" "ln" "st" ...
  .. ..$ suffix  : chr [1:33361] "-" "-" "-" "-" ...
  .. ..$ number  : int [1:33361] 1 1 1 1 1 1 1 1 1 1 ...
  .. ..$ month   : Ord.factor w/ 8 levels "january"<"february"<..: 1 1 1 1 1 1 1 1 1 1 ...
  .. ..$ day 

ERROR: Error in eval(expr, envir, enclos): could not find function "readOGR"


### CRS:
--------
#### spTransforms: To project spatial points from one geographic space to another:

CRS: The CRS system in R has various attributes such as projection, datum, and ellipsoid.
* Projection: Information about all different projects can be found out by executing the projInfo(type = "proj") in the R interpreter. Some of them are "** lonlat (for geographical coordinate), Mercator (For a planner projection), **. Projections account for all the different projections the coordinate points can be represented in.
* Datum: This contains a list of all the CRS such as ** WGS84 (CRS used by google earth), NAD83 (used by US/Canadian federal agencies) ** and their description. Basically it defines the origin and orientation of the coordinate axes (as well the size/shape of Earth). The datum
* ellipsoid (): Describes the shape of the earth. The ellipsoid is an approximation of earths shape and does not fit the Earth perfectly. There are different ellipsoids in use, some are designed to fit the whole Earth (WGS84, GRS80) and some are designed to fit a local region (NAD27). Local ellipses can be more accurate for the area they were designed for, but are not useful in other parts of the world. The modern trend is to use a global ellipsoid for compatibility, such as WGS84. 

**Note: The datum always specifies the ellipsoid that is used, but the ellipsoid does not specify the datum! Datums are based on specific ellipsoids and sometimes have the same name as the ellipsoid. The coordinate system such as WGS84, NAD83, and all geographic coordinate systems and are unprojected, hence they are good at preserving distances distances.
**


### EPSG Geodesic parameter dataset:
---------
The EPSG Geodetic Parameter Dataset is a structured dataset of Coordinate Reference Systems and Coordinate Transformations, accessible through this online registry (www.epsg-registry.org) or, as a downloadable zip files, through IOGP's EPSG home page at www.epsg.org. The geographic coverage of the data is worldwide, but it is stressed that the dataset does not and cannot record all possible geodetic parameters in use around the world. 
In R the details of a EPSG code can be determined by using **CRS("+init=epsg:4326")**. This reults is output such as **CRS arguments:
 +init=epsg:4326 +proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0 **
 
 Some comonly used EPSG code.
 
   * WGS84 (EPSG: 4326) --> For Latitude and Longitude
   * UTM, Zone 10 (EPSG: 32610) -> Projected Easting and northing
   * Mercator (EPSG: 3857) --> Tiles from Google Maps, Open Street Maps, Stamen Maps
   
    
### Projected Surface:
---------
Projecting a spherical surface on a flat plane is not trivial. Often on doing so we have to make some tradeoffs between getting preciseness for distance and direction.
For Example.
* Mercator preserves angles and direction and is useful for navigation. But distances and areas are distorted, especially near the polar regions. **UTM (Universal Transverse Mercator)**: The UTM projection is more commonly used in research because it is found to be more accurate locally.


In [6]:
data(state)

In [25]:
states <- data.frame(state.x77, state.center)
states <- states[states$x > -121,]
head(states)
coordinates(states) <- c("x", "y")
proj4string(states) <- CRS("+proj=longlat +ellps=clrk66")
summary(states)
state.merc <- spTransform(states,
 CRS=CRS("+proj=merc +ellps=GRS80 +units=us-mi"))
summary(state.merc)

,Population,Income,Illiteracy,Life.Exp,Murder,HS.Grad,Frost,Area,x,y
Alabama,3615,3624,2.1,69.05,15.1,41.3,20,50708,-86.7509,32.5901
Arizona,2212,4530,1.8,70.55,7.8,58.1,15,113417,-111.6250,34.2192
Arkansas,2110,3378,1.9,70.66,10.1,39.9,65,51945,-92.2992,34.7336
California,21198,5114,1.1,71.71,10.3,62.6,20,156361,-119.7730,36.5341
Colorado,2541,4884,0.7,72.06,6.8,63.9,166,103766,-105.5130,38.6777
Connecticut,3100,5348,1.1,72.48,3.1,56.0,139,4862,-72.3573,41.5928


Object of class SpatialPointsDataFrame
Coordinates:
        min      max
x -120.0680 -68.9801
y   27.8744  47.4231
Is projected: FALSE 
proj4string : [+proj=longlat +ellps=clrk66]
Number of points: 48
Data attributes:
   Population        Income       Illiteracy       Life.Exp    
 Min.   :  376   Min.   :3098   Min.   :0.500   Min.   :67.96  
 1st Qu.: 1188   1st Qu.:3964   1st Qu.:0.600   1st Qu.:70.13  
 Median : 2980   Median :4488   Median :0.900   Median :70.67  
 Mean   : 4398   Mean   :4386   Mean   :1.148   Mean   :70.85  
 3rd Qu.: 5064   3rd Qu.:4768   3rd Qu.:1.450   3rd Qu.:71.88  
 Max.   :21198   Max.   :5348   Max.   :2.800   Max.   :72.96  
     Murder          HS.Grad          Frost             Area       
 Min.   : 1.400   Min.   :37.80   Min.   : 11.00   Min.   :  1049  
 1st Qu.: 4.275   1st Qu.:47.70   1st Qu.: 68.75   1st Qu.: 38762  
 Median : 6.850   Median :53.05   Median :114.50   Median : 54277  
 Mean   : 7.321   Mean   :52.64   Mean   :105.65   Mean   : 61

ERROR: Error in spTransform(states, CRS = CRS("+proj=merc +ellps=GRS80 +units=us-mi")): package rgdal is required for spTransform methods
